In [1]:
uv pip install litellm

Audited 1 package in 42ms
Note: you may need to restart the kernel to use updated packages.


In [1]:
from litellm import completion


response = completion(
    model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
    performanceConfig={"latency": "optimized"},
)

print(response["choices"][0]["message"]["content"])

The capital of France is Paris.


In [ ]:
from langchain_aws import ChatBedrockConverse
import boto3

bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name='us-east-1',
)

llm = ChatBedrockConverse(
    model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.1,
    max_tokens=None,
    client=bedrock_client,
    performance_config={"latency": "optimized"}
)


In [23]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'aaf6593c-246a-4404-8fe8-1600d6d50c89', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 01 Aug 2025 16:54:20 GMT', 'content-type': 'application/json', 'content-length': '251', 'connection': 'keep-alive', 'x-amzn-requestid': 'aaf6593c-246a-4404-8fe8-1600d6d50c89'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1162]}, 'performanceConfig': {'latency': 'optimized'}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--62fd563b-10ff-4592-9362-de0e728f25e0-0', usage_metadata={'input_tokens': 25, 'output_tokens': 11, 'total_tokens': 36, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [3]:
from crewai import LLM, Agent, Task, Crew
from crewai.utilities.events import LLMStreamChunkEvent
from crewai.utilities.events.base_event_listener import BaseEventListener

class MyCustomListener(BaseEventListener):
    def setup_listeners(self, crewai_event_bus):
        @crewai_event_bus.on(LLMStreamChunkEvent)
        def on_llm_stream_chunk(source, event):
            if researcher.id == event.agent_id:
                print("\n==============\n Got event:", event, "\n==============\n")


my_listener = MyCustomListener()

llm = LLM(model="us.anthropic.claude-3-5-haiku-20241022-v1:0", temperature=0, stream=True)

researcher = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    llm=llm,
)

search = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=researcher,
)

crew = Crew(agents=[researcher], tasks=[search])

result = crew.kickoff(
    inputs={"question": "What is the capital of Spain?"}
)
print(result)

Thought
 Got event: timestamp=datetime.datetime(2025, 8, 8, 18, 22, 56, 631662, tzinfo=datetime.timezone.utc) type='llm_stream_chunk' source_fingerprint=None source_type=None fingerprint_metadata=None task_name=None task_id=UUID('d7fe9b5e-808d-48a6-8882-858891a355b9') agent_id=UUID('77237d1a-c2a8-46cb-ac48-f1776a2c630b') agent_role='About User' chunk='Thought' tool_call=None 


 Got event: timestamp=datetime.datetime(2025, 8, 8, 18, 22, 56, 631662, tzinfo=datetime.timezone.utc) type='llm_stream_chunk' source_fingerprint=None source_type=None fingerprint_metadata=None task_name=None task_id=UUID('d7fe9b5e-808d-48a6-8882-858891a355b9') agent_id=UUID('77237d1a-c2a8-46cb-ac48-f1776a2c630b') agent_role='About User' chunk='Thought' tool_call=None 


 Got event: timestamp=datetime.datetime(2025, 8, 8, 18, 22, 56, 631662, tzinfo=datetime.timezone.utc) type='llm_stream_chunk' source_fingerprint=None source_type=None fingerprint_metadata=None task_name=None task_id=UUID('d7fe9b5e-808d-48a6-8882-